<a href="https://colab.research.google.com/github/CG-94/WK_4_IP_Autolib/blob/main/Moringa_Data_Science_Prep_W4_Independent_Project_2021_03_Christabel_Gichobi_Python_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this week's independent project, you will be working as a data scientist working for an electric car-sharing service company. You have been tasked to process stations data to understand electric car usage over time by solving for the following research question;

Research Question

*   Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.

#1.0 Loading Datasets

In [57]:
import numpy as np  
import pandas as pd
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [ ]:
url = 'http://bit.ly/autolib_dataset'
Autolib = pd.read_csv(url)

In [58]:
#Now let's preview the dataset

Autolib.head(5)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23


In [59]:
# Accessing information about our dataset;

Autolib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

In [60]:
# Now let's preview the shape of our dataframe

Autolib.shape

(5000, 25)

# 1.1 Data Cleaning

In [61]:
# First we'll find if there are any missing values in our data and if so
#How many missing values are there?

Autolib.isna().sum().sum()

9842

In [62]:
# Now we drop the columns that don't add any value to our dataset
# We will then preview the shape of our new dataset

Autolib2=Autolib.drop(columns=['Displayed comment','Public name','Address','Scheduled at' ,'Geo point','Subscription status' ])
Autolib2.shape

(5000, 19)

In [63]:
#Now that the data is clean, we will create a new column with the timestamp for our analysis.
#We will then preview our dataset

Autolib2['Timestamp']= pd.to_datetime(Autolib2[['year', 'month', 'day', 'hour', 'minute']])
Autolib_Timestamp=Autolib2
Autolib_Timestamp.head(5)

,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,ID,Kind,Postal code,Rental status,Slots,Station type,Status,year,month,day,hour,minute,Timestamp
0,0,0,0,0,0,nonexistent,Paris,paris-suffren-2,STATION,75015,operational,2,station,ok,2018,4,8,11,43,2018-04-08 11:43:00
1,6,6,0,0,0,operational,Paris,paris-raymondlosserand-145,STATION,75014,operational,0,station,ok,2018,4,6,7,24,2018-04-06 07:24:00
2,3,3,0,2,0,operational,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,operational,1,station,ok,2018,4,3,20,14,2018-04-03 20:14:00
3,3,3,1,0,1,operational,Paris,paris-eugeneoudine-51,STATION,75013,operational,2,station,ok,2018,4,4,4,37,2018-04-04 04:37:00
4,3,3,0,0,0,nonexistent,Paris,paris-portedechamperret-6,PARKING,75017,operational,3,station,ok,2018,4,8,17,23,2018-04-08 17:23:00


# 1.2 Data Analysis

In [64]:
# To determine the most popular hour, first we need to know which cars were dropped off
# which were picked
# We will create a new dataframe for this with only the bluecar counter

Blue_Cars= Autolib_Timestamp.loc[:,['City','ID','Postal code','Bluecar counter','Timestamp','hour']]
Blue_Cars.head(10)

,City,ID,Postal code,Bluecar counter,Timestamp,hour
0,Paris,paris-suffren-2,75015,0,2018-04-08 11:43:00,11
1,Paris,paris-raymondlosserand-145,75014,6,2018-04-06 07:24:00,7
2,Le Bourget,lebourget-johnfitzgeraldkennedy-2,93350,3,2018-04-03 20:14:00,20
3,Paris,paris-eugeneoudine-51,75013,3,2018-04-04 04:37:00,4
4,Paris,paris-portedechamperret-6,75017,3,2018-04-08 17:23:00,17
5,Paris,paris-voltaire-8,75011,0,2018-04-06 07:02:00,7
6,Paris,paris-citroencevennes-parking,75015,0,2018-04-08 18:20:00,18
7,Nanterre,nanterre-luaps-17,92000,3,2018-04-04 22:13:00,22
8,Paris,paris-jeanmoulin-34,75014,1,2018-04-02 22:58:00,22
9,Paris,paris-anvers-parking,75009,6,2018-04-04 15:02:00,15


In [65]:
# First we wil sort our dataset by timestamp
# We then create a new entry showing if it was a pick up or drop off;
# This we do by getting the difference of the Bluecar counter rows. If there are more cars, there was a drop off
# Less cars means pick up

Blue_Cars=Blue_Cars.sort_values(by='Timestamp',ascending=True)
Blue_Cars['Incremental'] = Blue_Cars['Bluecar counter'].diff()
Blue_Cars= Blue_Cars[Blue_Cars.Incremental != 0]
Blue_Cars['Pick_Up/Drop_Off']=np.where(Blue_Cars['Incremental']>0, 'Drop', 'Pick')
Blue_Cars.head(10)

,City,ID,Postal code,Bluecar counter,Timestamp,hour,Incremental,Pick_Up/Drop_Off
1046,Paris,paris-monge-112,75005,0,2018-04-01 00:00:00,0,NaN,Pick
2059,Paris,paris-mozart-2,75016,1,2018-04-01 00:03:00,0,1.0,Drop
4453,Paris,paris-neuvesaintpierre-2,75004,0,2018-04-01 00:04:00,0,-1.0,Pick
584,Ãpinay-sur-Seine,epinaysurseine-avenir-1,93800,7,2018-04-01 00:05:00,0,7.0,Drop
4357,Paris,paris-margueritelong-11,75017,3,2018-04-01 00:10:00,0,-4.0,Pick
3652,Paris,paris-raymondpitet-7,75017,4,2018-04-01 00:11:00,0,1.0,Drop
3400,Garches,garches-degaulle-168,92380,5,2018-04-01 00:16:00,0,1.0,Drop
4792,Paris,paris-emileaugier-20,75016,3,2018-04-01 00:19:00,0,-2.0,Pick
2731,SÃ¨vres,sevres-granderue-123,92310,1,2018-04-01 00:20:00,0,-2.0,Pick
3583,Paris,paris-flandre-169,75019,3,2018-04-01 00:20:00,0,2.0,Drop


# Exporting cleaned and Analysed Data to CSV

In [66]:
# We will now export our new data to csv

Blue_Cars.to_csv('Blue_Cars.csv')


In [67]:
# Using sql, we will read our new csv file and answer our question.

with open('Blue_Cars.csv','r') as f:
    Blue_Cars2 = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS Blue_Cars2;
%sql PERSIST Blue_Cars2;

%sql SELECT * FROM Blue_Cars2 LIMIT 5

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


index,City,ID,Postal code,Bluecar counter,Timestamp,hour,Incremental,Pick_Up/Drop_Off
1046,Paris,paris-monge-112,75005,0,2018-04-01 00:00:00,0,None,Pick
2059,Paris,paris-mozart-2,75016,1,2018-04-01 00:03:00,0,1.0,Drop
4453,Paris,paris-neuvesaintpierre-2,75004,0,2018-04-01 00:04:00,0,-1.0,Pick
584,Ãpinay-sur-Seine,epinaysurseine-avenir-1,93800,7,2018-04-01 00:05:00,0,7.0,Drop
4357,Paris,paris-margueritelong-11,75017,3,2018-04-01 00:10:00,0,-4.0,Pick


# Using sql to categorize our data

In [ ]:
# First we create a new table with Paris Pick Ups only
%%sql
CREATE TABLE Paris_BlueCars2 AS
SELECT City, ID, Timestamp, "Pick_UP/Drop_Off", hour FROM Blue_Cars2
WHERE City = 'Paris' AND "Pick_UP/Drop_Off" = 'Pick';

In [69]:
# Previewing our new table;

%%sql
SELECT * FROM Paris_BlueCars2 LIMIT 5

 * sqlite://
Done.


City,ID,Timestamp,Pick_Up/Drop_Off,hour
Paris,paris-monge-112,2018-04-01 00:00:00,Pick,0
Paris,paris-neuvesaintpierre-2,2018-04-01 00:04:00,Pick,0
Paris,paris-margueritelong-11,2018-04-01 00:10:00,Pick,0
Paris,paris-emileaugier-20,2018-04-01 00:19:00,Pick,0
Paris,paris-villette-55,2018-04-01 00:35:00,Pick,0


In [70]:
# Now to get the most popular hour during the day;

%%sql
SELECT hour, count(hour) AS Popular
FROM Paris_BlueCars2
WHERE hour < 18
GROUP BY hour
ORDER BY Popular DESC;

 * sqlite://
Done.


hour,Popular
6,62
7,61
3,60
2,59
8,58
12,58
14,58
16,58
17,58
10,57
